# Intro to GraphX

Useful documentation:

 - https://spark.apache.org/docs/latest/graphx-programming-guide.html
 - https://spark.apache.org/docs/latest/api/java/org/apache/spark/graphx/Pregel.html
 - https://docs.databricks.com/spark/latest/graph-analysis/graph-analysis-graphx-tutorial.html 

Welcome to Scala!

In [1]:
val x:Int = 0

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1670079416392_0001,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

x: Int = 0


In [2]:
println("x = " + x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

x = 0


In [3]:
import org.apache.spark.graphx._

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.graphx._


## Creating graphs
Add some vertices:

In [4]:
val myVertices = sc.makeRDD(Array(
    (1L, "Alice"), 
    (2L, "Bob"),
    (3L, "Charlie"), 
    (4L, "John Doe"), 
    (5L, "Eve")
))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

myVertices: org.apache.spark.rdd.RDD[(Long, String)] = ParallelCollectionRDD[0] at makeRDD at <console>:28


Add edges with attributes:

In [5]:
val myEdges = sc.makeRDD(Array(
    Edge(1L, 2L, "is-friends-with"),
    Edge(2L, 3L, "is-friends-with"), 
    Edge(3L, 4L, "is-friends-with"),
    Edge(4L, 5L, "follows"), 
    Edge(3L, 5L, "follows")
))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

myEdges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = ParallelCollectionRDD[1] at makeRDD at <console>:28


Create a graph:

In [6]:
val myGraph = Graph(myVertices, myEdges)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

myGraph: org.apache.spark.graphx.Graph[String,String] = org.apache.spark.graphx.impl.GraphImpl@4cf57931


Look at vertices, edges and triplets:

In [7]:
myGraph.vertices.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res2: Array[(org.apache.spark.graphx.VertexId, String)] = Array((1,Alice), (2,Bob), (3,Charlie), (4,John Doe), (5,Eve))


In [8]:
myGraph.edges.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res3: Array[org.apache.spark.graphx.Edge[String]] = Array(Edge(1,2,is-friends-with), Edge(2,3,is-friends-with), Edge(3,4,is-friends-with), Edge(4,5,follows), Edge(3,5,follows))


In [9]:
myGraph.triplets.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res4: Array[org.apache.spark.graphx.EdgeTriplet[String,String]] = Array(((1,Alice),(2,Bob),is-friends-with), ((2,Bob),(3,Charlie),is-friends-with), ((3,Charlie),(4,John Doe),is-friends-with), ((4,John Doe),(5,Eve),follows), ((3,Charlie),(5,Eve),follows))


Iterate over all triplets:

In [10]:
myGraph.triplets.map(
  triplet => triplet.srcAttr + " " + triplet.attr + " " + triplet.dstAttr
).collect.foreach(println(_))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Alice is-friends-with Bob
Bob is-friends-with Charlie
Charlie is-friends-with John Doe
John Doe follows Eve
Charlie follows Eve


## Simple graph analysis
Compute pagerank:

In [11]:
val myPageRankGraph = myGraph.pageRank(0.0001)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

myPageRankGraph: org.apache.spark.graphx.Graph[Double,Double] = org.apache.spark.graphx.impl.GraphImpl@320b703d


In [12]:
myPageRankGraph.vertices.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res6: Array[(org.apache.spark.graphx.VertexId, Double)] = Array((1,0.4390416708169825), (2,0.8122270910114175), (3,1.1294346981766874), (4,0.9190514175420748), (5,1.7002451224528383))


In [13]:
myGraph.pageRank(0.0001).vertices.join(myVertices).collect.foreach(println)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1,(0.4390416708169825,Alice))
(2,(0.8122270910114175,Bob))
(3,(1.1294346981766874,Charlie))
(4,(0.9190514175420748,John Doe))
(5,(1.7002451224528383,Eve))


In [41]:
myGraph.inDegrees.foreach(x => println(x._1 + " has " + x._2 + " in degrees."))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Pregel program example

Goal: collect all the names of friends and followers in all the vertices. 
In every step all vertices send their names to all their connections, and merge together.

Initialize the graph such that all vertices start with empty set:


In [15]:
val initialGraph = myGraph.mapVertices((_,v) => Set(v))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

initialGraph: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@70e07188


In [16]:
initialGraph.vertices.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res9: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob)), (3,Set(Charlie)), (4,Set(John Doe)), (5,Set(Eve)))


Run Pregel for 1 step:

In [17]:
val g = Pregel(initialGraph,                                   //  Graph<VD,ED> graph
               Set[String](),                                  //  A initialMsg
               1,                                              //  int maxIterations,
               activeDirection = EdgeDirection.Out)(           //  EdgeDirection activeDirection,
               (id, value, message) => value union message,    //  scala.Function3<Object,VD,A,VD> vprog
               triplet =>                                      //  scala.Function1<EdgeTriplet<VD,ED>,scala.collection.Iterator<scala.Tuple2<Object,A>>> sendMsg,
                    Iterator((triplet.dstId, triplet.srcAttr)),
               (a,b) => a union b                              //  mergeMsg
              )              

g.vertices.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

g: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@723b561c
res11: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob, Alice)), (3,Set(Charlie, Bob)), (4,Set(John Doe, Charlie)), (5,Set(Eve, Charlie, John Doe)))


Run Pregel for 2 steps:

In [18]:
val g = Pregel(initialGraph,                                   //  Graph<VD,ED> graph
               Set[String](),                                  //  A initialMsg
               2,                                              //  int maxIterations,
               activeDirection = EdgeDirection.Out)(           //  EdgeDirection activeDirection,
               (id, value, message) => value union message,    //  scala.Function3<Object,VD,A,VD> vprog
               triplet =>                                      //  scala.Function1<EdgeTriplet<VD,ED>,scala.collection.Iterator<scala.Tuple2<Object,A>>> sendMsg,
                    Iterator((triplet.dstId, triplet.srcAttr)),
               (a,b) => a union b                              //  mergeMsg
              )              

g.vertices.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

g: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@45c03434
res13: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob, Alice)), (3,Set(Charlie, Bob, Alice)), (4,Set(John Doe, Charlie, Bob)), (5,Set(Eve, Charlie, John Doe, Bob)))


Run Pregel for 3 steps:

In [19]:
val g = Pregel(initialGraph,                                   //  Graph<VD,ED> graph
               Set[String](),                                  //  A initialMsg
               3,                                              //  int maxIterations,
               activeDirection = EdgeDirection.Out)(           //  EdgeDirection activeDirection,
               (id, value, message) => value union message,    //  scala.Function3<Object,VD,A,VD> vprog
               triplet =>                                      //  scala.Function1<EdgeTriplet<VD,ED>,scala.collection.Iterator<scala.Tuple2<Object,A>>> sendMsg,
                    Iterator((triplet.dstId, triplet.srcAttr)),
               (a,b) => a union b                              //  mergeMsg
              )              

g.vertices.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

g: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@542bceb0
res15: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob, Alice)), (3,Set(Charlie, Bob, Alice)), (4,Set(John Doe, Charlie, Bob, Alice)), (5,Set(Eve, Bob, Alice, John Doe, Charlie)))


The same but with more explicit type definitions:

In [20]:

// Vertex program - a function to apply to each vertex on a received message
def vprog(id:VertexId, value:Set[String], message:Set[String]): Set[String] = {
    value union message
}

// Send function: 
def sendMsg(et:EdgeTriplet[Set[String],String]): Iterator[Tuple2[VertexId,Set[String]]] = {                 
                    Iterator((et.dstId, et.srcAttr))
}

def mergeMsg(a:Set[String],b:Set[String]): Set[String] = {
    a union b
} 

val g = Pregel(initialGraph,                                 //  Graph<VD,ED> graph
               Set[String](),                                //  A initialMsg
               4,                                            //  int maxIterations,
               activeDirection = EdgeDirection.Out)(         //  EdgeDirection activeDirection,
               vprog,                                        //  scala.Function3<Object,VD,A,VD> vprog
               sendMsg ,                                     //  scala.Function1<EdgeTriplet<VD,ED>,scala.collection.Iterator<scala.Tuple2<Object,A>>> sendMsg,
               mergeMsg                                      //  mergeMsg: (A, A) => A
              )              

g.vertices.collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

vprog: (id: org.apache.spark.graphx.VertexId, value: Set[String], message: Set[String])Set[String]
sendMsg: (et: org.apache.spark.graphx.EdgeTriplet[Set[String],String])Iterator[(org.apache.spark.graphx.VertexId, Set[String])]
mergeMsg: (a: Set[String], b: Set[String])Set[String]
g: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@41e524a2
res22: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob, Alice)), (3,Set(Charlie, Bob, Alice)), (4,Set(John Doe, Charlie, Bob, Alice)), (5,Set(Eve, Bob, Alice, John Doe, Charlie)))


# Homework

## 1. Wczytać graf Facebooka, krawędzie z pliku musae_facebook_edges.csv, atrybuty page_name oraz page_type z pliku musae_facebook_target.csv.  Policzyć liczbę krawędzi i wierzchołków. (1 p.)

In [21]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD


In [ ]:
// val sc = (
//     SparkContext
//     .getOrCreate(
//         new SparkConf()
//         .setAppName("GraphX lab")
//         .setMaster("local[*]")
//     )
// )

val vertices: RDD[(Long, (String, String))] = (
    sc.textFile("https://e-d2wt81wy42gb0fzz01ci6f1y5.emrnotebooks-prod.us-east-1.amazonaws.com/e-D2WT81WY42GB0FZZ01CI6F1Y5/lab/tree/musae_facebook_target.csv")
    .map(line => {
        val x = line.split(",");
        (x(0).toLong, (x(2), x(3)))
    })
)

val edges: RDD[Edge[String]] = (
    sc.textFile("https://e-d2wt81wy42gb0fzz01ci6f1y5.emrnotebooks-prod.us-east-1.amazonaws.com/e-D2WT81WY42GB0FZZ01CI6F1Y5/lab/tree/musae_facebook_edges.csv")
    .map(line => {
        val x = line.split(",");
        Edge(x(0).toLong, x(1).toLong, "")
    })
)

val default_node = ("", "")

val graph: Graph[(String, String), String] = Graph(vertices, edges, default_node)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
graph.vertices.count()


In [ ]:
graph.edges.count()


## 2. Sprawdzić czy graf jest spójny. Czy dwa podgrafy utworzone dla typów strony governmental organizations oraz television shows też są spójne? (1 p.)

In [ ]:

def isConnected(graph: Graph[(String, String), String]): Boolean = {
    graph.vertices.count() == graph.connectedComponents().vertices.count()
}

In [ ]:
println(isConnected(graph))


In [ ]:
val government_graph = graph.subgraph(
    vpred = { case (id, (page_name, page_type)) => page_type == "government" }
)

println(isConnected(government_graph))

In [ ]:

val tvshow_graph = graph.subgraph(
    vpred = { case (id, (page_name, page_type)) => page_type == "tvshow" }
)

println(isConnected(tvshow_graph))


## 3. Spośród 1000 stron o najwyższym PageRank znaleźć 50 takich (wypisać page_name i page_type), które mają najmniej połączeń oraz 50 o największej liczbie połączeń. Który typ strony był dominujący w każdej z tych kategorii? Narysować wykres zależności PageRank od liczby krawędzi dla wierzchołków (scatter plot) (2 p.)

In [ ]:
val ranks = graph.pageRank(0.0001).vertices

val rankedVertexDegrees = ranks.join(graph.degrees)

In [ ]:
rankedVertexDegrees.takeSample(false, 3)


In [ ]:
val graphWithRankedDegrees = graph.outerJoinVertices(rankedVertexDegrees) {
    (_, data, vertexWithRank) => (
        data._1,  // page_name
        data._2,  // page_type
        vertexWithRank.getOrElse((0.0, 0))._1,  // PageRank value
        vertexWithRank.getOrElse((0.0, 0))._2   // degree
    )
}


In [ ]:
graphWithRankedDegrees.vertices.takeSample(false, 3)

In [ ]:
val top_50 = (
    graphWithRankedDegrees.vertices
    .top(50)
    .map(v => (v._2._2, v._2._1))  // degree, page_type, page_name
    .sorted
)


val last_50 = (
    graphWithRankedDegrees.vertices
    .takeOrdered(50)
    .map(v => (v._2._2, v._2._1))  // degree, page_type, page_name
    .sorted
)


In [ ]:
for (v <- top_50)
    println(v)


In [ ]:
for (v <- last_50)
    println(v)

In [ ]:
val rankedVertexDegreesValues = (
    rankedVertexDegrees
    .map(v => (v._2._1, v._2._2))
)

rankedVertexDegreesValues.takeSample(false, 3)


In [ ]:

import java.io._

val file = "degree_pagerank.txt"
val writer = new BufferedWriter(new OutputStreamWriter(new FileOutputStream(file)))

writer.write(rankedVertexDegreesValues.collect().mkString(","))
writer.close()

In [ ]:
%%python
from tempfile import NamedTemporaryFile

import matplotlib.pyplot as plt
from IPython.display import Image

with open("degree_pagerank.txt") as file:
    text = file.read()

tuples = [tup[:-1] for tup in text.split(",(")]
tuples[0] = tuples[0][1:]
tuples = [tup.split(",") for tup in tuples]
tuples = [(int(tup[1]), float(tup[0])) for tup in tuples]
tuples.sort()

degrees = [degree for pagerank_value, degree in tuples]
pagerank_values = [pagerank_value for pagerank_value, degree in tuples]

plt.clf()
plt.scatter(degrees, pagerank_values, c="b")
plt.title("PageRank value per node degree")

with NamedTemporaryFile(suffix=".png", delete=False) as file:
    plt.savefig(file.name)
    retval = Image(filename=file.name)

## 4. Korzystając z Pregel API zaimplementować następujący algorytm. W pierwszym kroku wybrana strona publikuje post fake news. W kolejnym kroku ten post publikowany jest przez 1/4 losowo wybranych kontaktów tej strony. W dalszych krokach, dla każdej strony, która opublikowała już ten post, losowo wybrane 1/4 jej kontaktów publikuje go u siebie. Pokazać jak zmienia się liczba stron które opublikowały post w zależności od liczby kroków. (2 p.)
Opcjonalnie: Sprawdzić powyższą zależność dla współczynnika innego niż 1/4.

In [ ]:
import scala.language.implicitConversions

implicit def bool2int(b:Boolean) = if (b) 1 else 0


def getPublishersNumber(graph: graphx.Graph[Boolean,String], maxIter: Int): Int = {
    val publishGraph = Pregel(
        graph,
        false,
        maxIter,
        activeDirection = EdgeDirection.Out
    )(
        (id, value, msg) => value || (msg && (scala.util.Random.nextFloat() < 0.333)),
        triplet => Iterator((triplet.dstId, triplet.srcAttr)),
        (a, b) => a || b
    )
    
    publishGraph.vertices.collect().map(v => v._2 * 1).sum
}

In [ ]:
val publishId = 1
val initialGraph = ((publishId: VertexId) => graph.mapVertices((id, _) => id == publishId))(publishId)

val publisherNumbers = for (maxIter <- 1 to 32) yield getPublishersNumber(initialGraph, maxIter)



In [ ]:
import java.io._

val file = "publisher_numbers.txt"
val writer = new BufferedWriter(new OutputStreamWriter(new FileOutputStream(file)))

writer.write(publisherNumbers.mkString(","))
writer.close()


In [ ]:
%%python
from tempfile import NamedTemporaryFile

import matplotlib.pyplot as plt
from IPython.display import Image

with open("publisher_numbers.txt") as file:
    text = file.read()

xs = list(range(1, 33))
ys = [int(val) for val in text.split(",")]

plt.clf()
plt.scatter(xs, ys, c="b")
plt.title("Numer of publishers per number of steps")

with NamedTemporaryFile(suffix=".png", delete=False) as file:
    plt.savefig(file.name)
    retval = Image(filename=file.name)

## 5. Narysować wykres rozkładu stopnia wierzchołków w grafie w skali logarytmicznej. Można skorzystać z funkcji obliczającej histogram dla RDD. Czy sieć jest bezskalowa (scale-free)? https://barabasi.com/f/623.pdf  (2 p.)

In [ ]:

val degrees = graph.degrees.map(v => v._2)

degrees.takeSample(false, 10)

In [ ]:

val (bins, values) = degrees.histogram(degrees.max)

In [ ]:
import java.io._

val file = "histogram_values.txt"
val writer = new BufferedWriter(new OutputStreamWriter(new FileOutputStream(file)))

writer.write(values.mkString(","))
writer.close()

In [ ]:
%%python
from tempfile import NamedTemporaryFile

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image

with open("histogram_values.txt") as file:
    text = file.read()

ys = np.array([int(val) for val in text.split(",")])
xs = list(range(1, len(ys) + 1))

plt.clf()
plt.scatter(np.log(xs), np.log(ys), color="b")
plt.title("Degree histogram")

with NamedTemporaryFile(suffix=".png", delete=False) as file:
    plt.savefig(file.name)
    retval = Image(filename=file.name)